In [10]:
from server_comms import ServerComms, ServerMessageTypes
import time
import threading
import json
from IPython.display import clear_output
import numpy as np
import utils
import random

In [23]:
server_message_types = ServerMessageTypes()

phew finished


In [17]:
class ApiRunner:
    def __init__(self):
        self.is_connected = True
        self.server = ServerComms("127.0.0.1", 8052)
            
    def send(self, type, payload=None):
        self.server.send_message(type, payload)
        
    def disconnect(self):
        self.is_connected = False
        self.server.server_socket.close()

In [18]:
api_runner = ApiRunner()

In [24]:
class TankRunner:
    def __init__(self, name, api_runner):
        self.name = name
        self.api_runner = api_runner
#         self.lock = threading.Lock()
        self.start()

    def start(self):
        self.has_ended = False
        self.state = dict(me={}, enemies={}, items={})
        self.spawn()
        self.receive_thread = threading.Thread(target=self.read)
        self.receive_thread.daemon = True
        self.receive_thread.start()
        
    def spawn(self):
        self.send(server_message_types.CREATETANK, dict(Name=self.name))
        
    def despawn(self):
        self.send(server_message_types.DESPAWNTANK)
        
    def read(self):
        while not self.has_ended:
#             self.lock.acquire()
            message = self.api_runner.server.read_message()
            self.update_state(message)
#             self.lock.release()
        print("phew finished")
    
    def update_state(self, message):
        if message == None: return
        if message['type'] == server_message_types.OBJECTUPDATE:
            data = message['data']
            if data['Type'] == 'Tank':
                if data['Name'] == self.name:
                    self.update_me(data)
                elif data['Name'] != self.name:
                    self.update_enemy(data)
            elif data['Type'] == 'AmmoPickup':
                self.update_item(data)
    
    def update_me(self, data):
        self.state['me'] = data
    
    def update_enemy(self, data):
        self.state['enemies'][data['Name']] = data
    
    def update_item(self, data):
        self.state['items'][data['Id']] = data
    
    def send(self, type, payload=None):
        self.api_runner.send(type, payload)
    
    def stop(self):
        self.has_ended = True
        self.despawn()
        
    def do_action(self, action_type):
        if action_type == 1:
            self.send(server_message_types.TOGGLEFORWARD)
        elif action_type == 2:
            self.send(server_message_types.TOGGLEREVERSE)
        elif action_type == 3:
            self.send(server_message_types.TOGGLELEFT)
        elif action_type == 4:
            self.send(server_message_types.TOGGLERIGHT)
        elif action_type == 5:
            self.send(server_message_types.TOGGLETURRETLEFT)
        elif action_type == 6:
            self.send(server_message_types.TOGGLETURRETRIGHT)
        elif action_type == 7:
            self.send(server_message_types.FIRE)
        elif action_type == 8:
            self.send(server_message_types.STOPMOVE)
        elif action_type == 9:
            self.send(server_message_types.STOPTURN)
        elif action_type == 10:
            self.send(server_message_types.STOPTURRET)
        elif action_type == 11:
            self.send(server_message_types.STOPALL)
        
    def calculate_reward(self):
        pass
    
    def get_closest_enemy(self):
        min_distance = -99999
        min_coords = (-99999, -99999)
        me_coords = (self.state['me']['X'], self.state['me']['Y'])
        for enemy in self.state['enemies'].values():
            enemy_coords = (enemy['X'], enemy['Y'])
            distance = utils.calculate_distance(me_coords, enemy_coords)
            if min_distance == -99999 or min_distance < distance:
                min_distance = distance
                min_coords = enemy_coords
        return min_coords, round(min_distance, 3)
    
    def get_number_of_enemies(self):
        return len(self.state['enemies'].keys())
    
    def get_state_details(self):
        if len(self.state['me'].keys()) == 0: return
        me_coords = [round(self.state['me']['X'],3), round(self.state['me']['Y'],3)]
        closest_enemy_coords, closest_enemy_distance = self.get_closest_enemy()
        closest_enemy_coords = (round(closest_enemy_coords[0], 3), round(closest_enemy_coords[1], 3))
        number_of_enemies = self.get_number_of_enemies()
        return None
    
    def step(self, action):
        self.do_action(action)
        time.sleep(0.1)
        state = self.get_state_details()
#         reward = self.calculate_reward()
        return state

In [26]:
for i in range(2):
    tank_runner = TankRunner("bruh{0}".format(i), api_runner)
    for i in range(10):
        action = random.randint(1,12)
        tank_runner.step(action)
        time.sleep(2)
    time.sleep(5)
    tank_runner.stop()

phew finished
phew finished
phew finished


In [21]:
tank_runner = TankRunner("bruh{0}".format(1), api_runner)

In [ ]:
tank_runner.step(2)
tank_runner.step(3)

In [ ]:
tank_runner.stop()